<a href="https://colab.research.google.com/github/gabrieldilay/TitanicAnaliseSupervisionada/blob/main/TitanicNaoSupervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
titanic_train_df = pd.read_csv('/content/drive/MyDrive/Data Science/train.csv',
                      sep=',', encoding='iso-8859-1')
titanic_test_df = pd.read_csv('/content/drive/MyDrive/Data Science/test.csv',
                      sep=',', encoding='iso-8859-1')
titanic_test_df_y = pd.read_csv('/content/drive/MyDrive/Data Science/gender_submission.csv',
                      sep=',', encoding='iso-8859-1')
titanic_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tratamento de Dados

In [5]:
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


*Podemos ver que faltam alguns valores nas colunas Age, Cabin e Embarked.*

In [6]:
titanic_train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

*Podemos ver a quantiade de valores faltantes em cada coluna.*

In [7]:
#Removendo as linhas da coluna Embarked com valores faltantes

titanic_train_df = titanic_train_df[~titanic_train_df['Embarked'].isnull()]

In [8]:
#Convertendo os valores da coluna Cabin para 0 e 1. 
#1 representa se um passageiro recebeu uma cabine e 0 se nao recebeu.

titanic_train_df['Cabin']= titanic_train_df['Cabin'].fillna(0)
titanic_train_df.loc[titanic_train_df['Cabin']!=0,'Cabin'] = 1
titanic_test_df['Cabin']= titanic_test_df['Cabin'].fillna(0)
titanic_test_df.loc[titanic_test_df['Cabin']!=0,'Cabin'] = 1

In [9]:
#Excluindo a tabela Age

titanic_train_df = titanic_train_df.drop(['Age'],axis=1)
titanic_test_df = titanic_test_df.drop(['Age'],axis=1)

In [10]:
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Ticket       889 non-null    object 
 8   Fare         889 non-null    float64
 9   Cabin        889 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(1), int64(5), object(5)
memory usage: 83.3+ KB


In [11]:
#Converter os dados do tipo de objeto para inteiro

titanic_train_df['Cabin'] = pd.to_numeric(titanic_train_df['Cabin'], errors='coerce')
titanic_test_df['Cabin'] = pd.to_numeric(titanic_test_df['Cabin'], errors='coerce')

In [12]:
#Converter valores da coluna Sex para 0 e 1

varlist =  ['Sex']

#Usando map() para achar valores da coluna
def binary_map(x):
    return x.map({"male": 1, "female": 0})

#Aplicando os valores 
titanic_train_df[varlist] = titanic_train_df[varlist].apply(binary_map)
titanic_test_df[varlist] = titanic_test_df[varlist].apply(binary_map)

In [13]:
#Usar replace() para trocar os valores da coluna Embarked com os seguintes valores

titanic_train_df['Embarked'].replace({'S':0, 'C':1, 'Q':2}, inplace=True)
titanic_test_df['Embarked'].replace({'S':0, 'C':1, 'Q':2}, inplace=True)

In [14]:
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    int64  
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Ticket       889 non-null    object 
 8   Fare         889 non-null    float64
 9   Cabin        889 non-null    int64  
 10  Embarked     889 non-null    int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 83.3+ KB


In [15]:
titanic_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Ticket       418 non-null    object 
 7   Fare         417 non-null    float64
 8   Cabin        418 non-null    int64  
 9   Embarked     418 non-null    int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 32.8+ KB


*Podemos ver que na tabela test ainda falta um valor na coluna Fare*

In [16]:
titanic_test_df = titanic_test_df.fillna(0)

In [18]:
titanic_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Ticket       418 non-null    object 
 7   Fare         418 non-null    float64
 8   Cabin        418 non-null    int64  
 9   Embarked     418 non-null    int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 32.8+ KB


***Agora todas as bases estão limpas e ajustadas.***